# Rebuild All Events
    This script builds/rebuilds all the events of Live Mouse Tracker
    You can also create your own events and add them to this process
    
    Approximate time is 1h of computation for a 24h database of 4 animals


# Parameters:
    This will compute automatically events for your database, and adjust settings for you.
    
    If you remove the automatic settings, you can force parameters:
        Set minT and maxT to process the database.
        Set windowT to divide the computation load in segment of that duration. Default value is 1 day.

In [1]:
import sys
sys.path.insert(1, "../")
from lmtanalysis.Measure import *

# set this to false if you want to set manual parameters.
automaticSettings = True

# Manual parameters:

''' minT and maxT to process the analysis (expressed in frame) '''
minT = 0
maxT = 3*oneDay
''' time window to compute the events (in frame). '''
windowT = 1*oneDay
''' speed up process '''
USE_CACHE_LOAD_DETECTION_CACHE = True

# Run this section to compute your database
    You will be prompt to provide a database

In [2]:
''' Created on 26 march 2019 @author: Fab '''

import sys
sys.path.insert(1, "../")
import sqlite3
from lmtanalysis.Animal import *
import matplotlib.pyplot as plt
from lmtanalysis.Event import *
from lmtanalysis.Measure import *
from lmtanalysis import BuildEventTrain3, BuildEventTrain4, BuildEventTrain2, BuildEventFollowZone, BuildEventRear5, BuildEventFloorSniffing,\
    BuildEventSocialApproach, BuildEventSocialEscape, BuildEventApproachContact,BuildEventOralOralContact,\
    BuildEventApproachRear, BuildEventGroup2, BuildEventGroup3, BuildEventGroup4, BuildEventOralGenitalContact, \
    BuildEventStop, BuildEventWaterPoint, \
    BuildEventMove, BuildEventGroup3MakeBreak, BuildEventGroup4MakeBreak,\
    BuildEventSideBySide, BuildEventSideBySideOpposite, BuildEventDetection,\
    BuildDataBaseIndex, BuildEventWallJump, BuildEventSAP,\
    BuildEventOralSideSequence, CheckWrongAnimal,\
    CorrectDetectionIntegrity, BuildEventNest4, BuildEventNest3, BuildEventFight, BuildEventGetAway
    
from psutil import virtual_memory

from tkinter.filedialog import askopenfilename
from lmtanalysis.TaskLogger import TaskLogger
import sys
import traceback
from lmtanalysis.FileUtil import getFilesToProcess
from lmtanalysis.EventTimeLineCache import flushEventTimeLineCache,\
    disableEventTimeLineCache


from lmtanalysis.EventTimeLineCache import EventTimeLineCached

def getNumberOfFrames(file):
    
    connection = sqlite3.connect( file )
    c = connection.cursor() 
    query = "SELECT MAX(FRAMENUMBER) FROM FRAME";
    c.execute( query )
    result = c.fetchall()
    numberOfFrames = int( result[0][0] )
    connection.close()
    
    return numberOfFrames

class FileProcessException(Exception):
    pass

eventClassList = [                  
                  BuildEventOralOralContact,
                  BuildEventOralGenitalContact,
                  BuildEventSideBySide,
                  BuildEventSideBySideOpposite,
                  BuildEventTrain2,                  
                  BuildEventTrain3,
                  BuildEventTrain4,
                  BuildEventMove,
                  BuildEventFollowZone,
                  BuildEventRear5,
                  BuildEventSocialApproach,
                  BuildEventGetAway,
                  BuildEventSocialEscape,
                  BuildEventApproachRear,
                  BuildEventGroup2,
                  BuildEventGroup3,
                  BuildEventGroup4,
                  BuildEventGroup3MakeBreak,
                  BuildEventGroup4MakeBreak,
                  BuildEventStop,
                  BuildEventWaterPoint,
                  BuildEventApproachContact,
                  BuildEventWallJump,
                  BuildEventSAP,
                  BuildEventOralSideSequence,
                  BuildEventNest3,
                  BuildEventNest4
                   ]

def flushEvents( connection ):
    
    print("Flushing events...")

    for ev in eventClassList:
        
        chrono = Chronometer( "Flushing event " + str(ev) )
        ev.flush( connection );      
        chrono.printTimeInS()
    

def processTimeWindow( connection, currentMinT , currentMaxT ):
    
    CheckWrongAnimal.check( connection, tmin=currentMinT, tmax=currentMaxT )
    
    # Warning: enabling this process (CorrectDetectionIntegrity) will alter the database permanently
    # CorrectDetectionIntegrity.correct( connection, tmin=0, tmax=maxT )
                            
    BuildEventDetection.reBuildEvent( connection, file, tmin=currentMinT, tmax=currentMaxT )

    animalPool = None
    
    flushEventTimeLineCache()
    
    if ( USE_CACHE_LOAD_DETECTION_CACHE ):
        print("Caching load of animal detection...")
        animalPool = AnimalPool( )
        animalPool.loadAnimals( connection )
        animalPool.loadDetection( start = currentMinT, end = currentMaxT )
        print("Caching load of animal detection done.")

    for ev in eventClassList:
        
        chrono = Chronometer( str( ev ) )
        ev.reBuildEvent( connection, file, tmin=currentMinT, tmax=currentMaxT, pool = animalPool )        
        chrono.printTimeInS()

def process( file ):

    print(file)
    
    if automaticSettings:
        print("Automatic settings.")
        windowT = 1*oneDay
        minT = 0
        maxT = getNumberOfFrames( file )
        print ( "Auto max set to" , maxT , "frames")
        
    
    chronoFullFile = Chronometer("File " + file )
    
    connection = sqlite3.connect( file )
    
    BuildDataBaseIndex.buildDataBaseIndex( connection, force=False )
        
    '''
    # build sensor data    
    animalPool = AnimalPool( )
    animalPool.loadAnimals( connection )
    animalPool.buildSensorData(file)
    '''
    
    currentT = minT

    try:

        flushEvents( connection )
        
        while currentT < maxT:
                        
            currentMinT = currentT
            currentMaxT = currentT+ windowT
            if ( currentMaxT > maxT ):
                currentMaxT = maxT
                
            chronoTimeWindowFile = Chronometer("File "+ file+ " currentMinT: "+ str(currentMinT)+ " currentMaxT: " + str(currentMaxT) );
            processTimeWindow( connection, currentMinT, currentMaxT )    
            chronoTimeWindowFile.printTimeInS()
            
            currentT += windowT

                        

        print("Full file process time: ")
        chronoFullFile.printTimeInS()
        
        TEST_WINDOWING_COMPUTATION = False
        
        if ( TEST_WINDOWING_COMPUTATION ):
                
            print("*************")
            print("************* TEST START SECTION")
            print("************* Test if results are the same with or without the windowing.")
            
            # display and record to a file all events found, checking with rolling idA from None to 4. Save nbEvent and total len
            
            eventTimeLineList = []
            
            eventList = getAllEvents( connection )
            file = open("outEvent"+str(windowT)+".txt","w")  
            file.write( "Event name\nnb event\ntotal duration" )
            
            for eventName in eventList:
                for idAnimalA in range( 0,5 ):                
                        idA = idAnimalA 
                        if idA == 0:
                            idA = None
                        timeLine = EventTimeLineCached( connection, file, eventName, idA,  minFrame=minT, maxFrame=maxT )
                        eventTimeLineList.append( timeLine )
                        file.write( timeLine.eventNameWithId+"\t"+str(len(timeLine.eventList))+"\t"+str(timeLine.getTotalLength())+"\n" )            
            
            file.close() 
    
            #plotMultipleTimeLine( eventTimeLineList )
            
            print("************* END TEST")
        
        
    except:
        
        exc_type, exc_value, exc_traceback = sys.exc_info()
        lines = traceback.format_exception(exc_type, exc_value, exc_traceback)
        error = ''.join('!! ' + line for line in lines)
        
        t = TaskLogger( connection )
        t.addLog( error )
        
        print( error, file=sys.stderr ) 
        
        raise FileProcessException()
            
def getAllEvents( connection ):
    
    query = "select name from event group by name order by name"
    c = connection.cursor()     
    c.execute( query )    
    all_rows = c.fetchall()        
    header = [ "Name" ]
    data =[]    
    for row in all_rows:
        data.append( row[0] )
    return data        

print("Code launched.")

mem = virtual_memory()
availableMemoryGB = mem.total / 1000000000
print( "Total memory on computer: (GB)", availableMemoryGB ) 

if availableMemoryGB < 10:
    print( "Not enough memory to use cache load of events.")
    disableEventTimeLineCache()

print("A window is popping (maybe hidden) asking for files to process...")
files = getFilesToProcess()

chronoFullBatch = Chronometer("Full batch" )    

if ( files != None ):

    for file in files:
        try:
            print ( "Processing file" , file )
            process( file )
        except FileProcessException:
            print ( "STOP PROCESSING FILE " + file , file=sys.stderr  )

        flushEventTimeLineCache()

chronoFullBatch.printTimeInS()
print( "*** ALL JOBS DONE ***")

Code launched.
Total memory on computer: (GB) 17.078222848
A window is popping (maybe hidden) asking for files to process...
Processing file C:/Users/Fab/Desktop/20180110_validation_2_ind_Experiment_0044_e.sqlite
C:/Users/Fab/Desktop/20180110_validation_2_ind_Experiment_0044_e.sqlite
Automatic settings.
Auto max set to 23522 frames
Creating lmtanalysis indexes...
CREATE INDEX `animalIndex` ON `ANIMAL` (`ID` );
index animalIndex already exists
CREATE INDEX `detectionIndex` ON `DETECTION` (`ID` ASC,`FRAMENUMBER` ASC);
index detectionIndex already exists
CREATE INDEX `detetIdIndex` ON `DETECTION` (`ID` ASC);
index detetIdIndex already exists
CREATE INDEX `detframenumberIndex` ON `DETECTION` (`FRAMENUMBER` ASC);
index detframenumberIndex already exists
CREATE INDEX `eventEndFrameIndex` ON `EVENT` (`ENDFRAME` ASC);
index eventEndFrameIndex already exists
CREATE INDEX `eventIndex` ON `EVENT` (`ID` ASC,`STARTFRAME` ASC,`ENDFRAME` ASC);
index eventIndex already exists
CREATE INDEX `eventStartF

 detections loaded in 0.06252765655517578 seconds.
Number of event:  9
Mean length of event:  2610.5555555555557
first event frame:  2
Keep previous entry.
Saving timeLine: Detection ( 9 events )
Detection
Event Detection created. eventNameWithId = Detection idA:2 idB:None idC:None idD:None loadEvent: False
processing animal ID: 2
SELECT FRAMENUMBER FROM DETECTION WHERE ANIMALID=2 AND FRAMENUMBER>=0 AND FRAMENUMBER<=23522
 detections loaded in 0.0156402587890625 seconds.
Number of event:  17
Mean length of event:  1377.0
first event frame:  16
Keep previous entry.
Saving timeLine: Detection ( 17 events )
Detection
Event Detection created. eventNameWithId = Detection idA:3 idB:None idC:None idD:None loadEvent: False
processing animal ID: 3
SELECT FRAMENUMBER FROM DETECTION WHERE ANIMALID=3 AND FRAMENUMBER>=0 AND FRAMENUMBER<=23522
 detections loaded in 0.0 seconds.
no event
Keep previous entry.
Saving timeLine: Detection ( 0 events )
Detection
Event Detection created. eventNameWithId = 

Contact  Id( 1 , None , None , None ) Min/maxFrame: ( 0 / 23522 ) Loaded ( 139  records loaded in  0.0 S )
Caching eventTimeLine
Stop  Id( 2 , None , None , None ) Min/maxFrame: ( 0 / 23522 ) Loaded ( 743  records loaded in  0.03126931190490723 S )
Detection  Id( 2 , None , None , None ) Min/maxFrame: ( 0 / 23522 ) Loaded ( 17  records loaded in  0.053375244140625 S )
Contact  Id( 2 , None , None , None ) Min/maxFrame: ( 0 / 23522 ) Loaded ( 139  records loaded in  0.015649795532226562 S )
Caching eventTimeLine
Event Move created. eventNameWithId = Move idA:1 idB:None idC:None idD:None loadEvent: False
Number of event:  702
Mean length of event:  14.183760683760683
first event frame:  2
Keep previous entry.
Saving timeLine: Move ( 702 events )
Event Move isolated created. eventNameWithId = Move isolated idA:1 idB:None idC:None idD:None loadEvent: False
Number of event:  680
Mean length of event:  13.555882352941177
first event frame:  2
Keep previous entry.
Saving timeLine: Move isolat

INSERT INTO LOG( process,version,date,tmin,tmax) VALUES ( 'Build Event Social Escape','0','2019-07-22 15:40:16','0','23522' );
Rebuild event finished.
[Chrono  <module 'lmtanalysis.BuildEventSocialEscape' from '..\\lmtanalysis\\BuildEventSocialEscape.py'>  ]  0.26561427116394043  seconds
Rear in contact  Id( 1 , None , None , None ) Min/maxFrame: ( 0 / 23522 ) Loaded ( 29  records loaded in  0.0 S )
Social approach  Id( 1 , 2 , None , None ) Min/maxFrame: ( 0 / 23522 ) Loaded ( 260  records loaded in  0.01562786102294922 S )
Caching eventTimeLine
Social approach  Id( 1 , 3 , None , None ) Min/maxFrame: ( 0 / 23522 ) Loaded ( 0  records loaded in  0.015644311904907227 S )
Caching eventTimeLine
Social approach  Id( 1 , 4 , None , None ) Min/maxFrame: ( 0 / 23522 ) Loaded ( 0  records loaded in  0.0 S )
Caching eventTimeLine
Rear in contact  Id( 2 , None , None , None ) Min/maxFrame: ( 0 / 23522 ) Loaded ( 16  records loaded in  0.015625 S )
Social approach  Id( 2 , 1 , None , None ) Min/

Group2
Event Group2 created. eventNameWithId = Group2 idA:2 idB:3 idC:None idD:None loadEvent: False
Group2
Event Group2 created. eventNameWithId = Group2 idA:2 idB:3 idC:None idD:None loadEvent: False
no event
Keep previous entry.
Saving timeLine: Group2 ( 0 events )
Group2
Event Group2 created. eventNameWithId = Group2 idA:2 idB:4 idC:None idD:None loadEvent: False
Group2
Event Group2 created. eventNameWithId = Group2 idA:2 idB:4 idC:None idD:None loadEvent: False
no event
Keep previous entry.
Saving timeLine: Group2 ( 0 events )
Group2
Event Group2 created. eventNameWithId = Group2 idA:3 idB:1 idC:None idD:None loadEvent: False
Group2
Event Group2 created. eventNameWithId = Group2 idA:3 idB:1 idC:None idD:None loadEvent: False
no event
Keep previous entry.
Saving timeLine: Group2 ( 0 events )
Group2
Event Group2 created. eventNameWithId = Group2 idA:3 idB:2 idC:None idD:None loadEvent: False
Group2
Event Group2 created. eventNameWithId = Group2 idA:3 idB:2 idC:None idD:None loadEven

Group3  Id( 2 , 3 , 1 , None ) Min/maxFrame: ( 0 / 23522 ) Loaded ( 0  records loaded in  0.015630245208740234 S )
Caching eventTimeLine
Group3  Id( 2 , 3 , 4 , None ) Min/maxFrame: ( 0 / 23522 ) Loaded ( 0  records loaded in  0.0 S )
Caching eventTimeLine
Group3  Id( 2 , 4 , 1 , None ) Min/maxFrame: ( 0 / 23522 ) Loaded ( 0  records loaded in  0.015621662139892578 S )
Caching eventTimeLine
Group3  Id( 2 , 4 , 3 , None ) Min/maxFrame: ( 0 / 23522 ) Loaded ( 0  records loaded in  0.015624046325683594 S )
Caching eventTimeLine
Group3  Id( 3 , 1 , 2 , None ) Min/maxFrame: ( 0 / 23522 ) Loaded ( 0  records loaded in  0.0 S )
Caching eventTimeLine
Group3  Id( 3 , 1 , 4 , None ) Min/maxFrame: ( 0 / 23522 ) Loaded ( 0  records loaded in  0.01562809944152832 S )
Caching eventTimeLine
Group3  Id( 3 , 2 , 1 , None ) Min/maxFrame: ( 0 / 23522 ) Loaded ( 0  records loaded in  0.0 S )
Caching eventTimeLine
Group3  Id( 3 , 2 , 4 , None ) Min/maxFrame: ( 0 / 23522 ) Loaded ( 0  records loaded in  0.0

Stop  Id( 1 , None , None , None ) Min/maxFrame: ( 0 / 23522 ) Loaded ( 693  records loaded in  0.015624761581420898 S )
Caching eventTimeLine
Contact  Id( 1 , None , None , None ) Loaded from cache ( 139  records. )
Stop  Id( 2 , None , None , None ) Min/maxFrame: ( 0 / 23522 ) Loaded ( 742  records loaded in  0.031256914138793945 S )
Caching eventTimeLine
Contact  Id( 2 , None , None , None ) Loaded from cache ( 139  records. )
Stop  Id( 3 , None , None , None ) Min/maxFrame: ( 0 / 23522 ) Loaded ( 0  records loaded in  0.015622854232788086 S )
Caching eventTimeLine
Contact  Id( 3 , None , None , None ) Loaded from cache ( 0  records. )
Stop  Id( 4 , None , None , None ) Min/maxFrame: ( 0 / 23522 ) Loaded ( 0  records loaded in  0.0 S )
Caching eventTimeLine
Contact  Id( 4 , None , None , None ) Loaded from cache ( 0  records. )
Stop isolated Stop in contact
Event Stop in contact created. eventNameWithId = Stop in contact idA:1 idB:None idC:None idD:None loadEvent: False
Number of ev